In [8]:
import tensorflow as tf
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import cv2 as cv

First we define some function that will be usefull for us.

In [68]:
# Constants
width = 224
height = 224

# Building the neural network
def buildSimpleNN():
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

    model.compile(optimizer='rmsprop', loss='mse')

    return model

# Training the neural network
def trainModel(X, Y, model):
    print('Training model...')
    model.fit(x=X[:amount], y=Y[:amount], batch_size=2, epochs=30, verbose=0)
    print('Model trained.')

# Get the luminance of the image aka the image in black and white
def getLFromLab(image_loaded):
    L_image = cv.cvtColor(np.float32(
        1.0/255 * image_loaded), cv.COLOR_RGB2LAB)[:, :, 0]
    L_image = L_image.reshape(1, width, height, 1)
    return L_image

# Get a image with colorized pixels
def getColorizeImage(image_name, model):
    #Get image data
    image_loaded = img_to_array(load_img(image_name))
    image_loaded = cv.resize(image_loaded, (width, height))
    image_loaded = np.array(image_loaded, dtype=float)
    #Get luminosity/Black&White image
    L_image = getLFromLab(image_loaded)
    #Predict
    output = model.predict(L_image)
    output = output * 128
    #Transform output to image
    cur = np.zeros((width, height, 3))
    cur[:,:,0] = L_image[0][:,:,0]
    cur[:,:,1:] = output[0]
    #Convert to BGR
    cv_result = cv.cvtColor(np.float32(cur),  cv.COLOR_LAB2BGR)
    cv_result = cv_result * 255
    #cv_result = cv_result.round(0)
    cv.imwrite("img_result_cv.png", cv_result)


In [10]:
# Get images
image = img_to_array(load_img('./input/images/woman.jpg'))
image = cv.resize(image, (400, 400))
image = np.array(image, dtype=float)

In [11]:
X = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,0]
Y = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [59]:
1.0/255*image[0][0][0]


0.6705882352941176

In [58]:
X[0][0][0][0]


87.79297

In [12]:
# Finish model
model = buildSimpleNN()

In [19]:
# Train model
trainModel(X, Y, model)

Training model...
Model trained.


In [27]:
from datetime import datetime

# Save model
model.save('model_'+datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+'.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, None, None, 8)     80        
                                                                 
 conv2d_10 (Conv2D)          (None, None, None, 8)     584       
                                                                 
 conv2d_11 (Conv2D)          (None, None, None, 16)    1168      
                                                                 
 conv2d_12 (Conv2D)          (None, None, None, 16)    2320      
                                                                 
 conv2d_13 (Conv2D)          (None, None, None, 32)    4640      
                                                                 
 conv2d_14 (Conv2D)          (None, None, None, 32)    9248      
                                                                 
 up_sampling2d_3 (UpSampling  (None, None, None, 32)  

In [62]:
getColorizeImage('./input/images/woman.jpg', model)


1/1 [==============================] - 0s 95ms/step


Use a model already compiled

In [101]:
from keras.models import load_model

model_big = load_model('model_big_2.h5')
getColorizeImage('./input/images/woman.jpg', model_big)

1/1 [==============================] - 0s 308ms/step


In [102]:
image_name = './input/images/woman.jpg'

#Get image data
image_loaded = img_to_array(load_img(image_name))
image_loaded = cv.resize(image_loaded, (width, height))
image_loaded = np.array(image_loaded, dtype=float)
#Get luminosity/Black&White image
L_image = getLFromLab(image_loaded)
#Predict
output_1 = model.predict(L_image)
output_2 = model_big.predict(L_image)


1/1 [==============================] - 0s 71ms/step


In [103]:
output_1[0][0][0]*128

array([ -6.6923175, -12.14242  ], dtype=float32)

In [104]:
output_2[0][0][0]*128

array([128., 128.], dtype=float32)

In [100]:
image_loaded[0][0][0]


170.52296447753906